I have the objective function to minimize:

$$
\min_{\Gamma, F_t} \sum_{i,t} (Y_{i,t} - X_{i,t} \Gamma F_t)^2 . \quad \forall i \in N_{co}, \forall t \in T
$$

After estimating $F_t$ correctly I will need to esimate $\Gamma$, explain to me if the following equation can correctly estimate $\Gamma$ and explain to me why

$$
\hat{\gamma} = \left( \sum_{i,t} (X_{i,t}' \otimes \hat{F}_t) (X_{i,t} \otimes \hat{F}_t') \right)^{-1} \left( \sum_{i,t} (X_{i,t}' \otimes \hat{F}_t) Y_{i,t} \right)
$$


In [1]:
import pandas as pd
import numpy as np
import scipy.linalg as sla
import scipy.sparse.linalg as ssla

import matplotlib.pyplot as plt
import seaborn as sns

from src.data_gen import data_gen, data_gen_xu

# Set the global font to be Times New Roman
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 8 
colors = sns.color_palette()
colors

[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (1.0, 0.4980392156862745, 0.054901960784313725),
 (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
 (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
 (0.5803921568627451, 0.403921568627451, 0.7411764705882353),
 (0.5490196078431373, 0.33725490196078434, 0.29411764705882354),
 (0.8901960784313725, 0.4666666666666667, 0.7607843137254902),
 (0.4980392156862745, 0.4980392156862745, 0.4980392156862745),
 (0.7372549019607844, 0.7411764705882353, 0.13333333333333333),
 (0.09019607843137255, 0.7450980392156863, 0.8117647058823529)]

In [2]:
# matrix left/right division (following MATLAB function naming)
_mldivide = lambda denom, numer: sla.lstsq(np.array(denom), np.array(numer))[0]
_mrdivide = lambda numer, denom: (sla.lstsq(np.array(denom).T, np.array(numer).T)[0]).T

In [3]:
# Number of factors and time periods
K = 3
T0 = 20
T1 = 10
# number of total covariates
L = 10
# Number of control and treated units

N_co = 45
N_tr = 5
# drift for treated units
drift = 2
# similarity parameter in Xu
w = 0.8

# random seed
np.random.seed(39)
# gen data with my data generating process
df = data_gen(T0, T1, N_co, N_tr, L, K, drift)

# gen data with Yiqing Xu data generating process
#df = data_gen_xu(T0, T1, N_co, N_tr, L, K, w)

In [4]:
# get a list of covariates
covariates = df.columns[7:].values

# build a function to prepare matrix
def prepare_matrix(df, id, time, outcome, covariates):
    Y = df.pivot(index=id, columns=time, values=outcome).values
    X = np.array([df.pivot(index=id, columns=time, values=var).values for var in covariates]).transpose(1, 2, 0)
    return Y, X

# prepare matrix
Y0, X0 = prepare_matrix(df.query("tr_group==0"), 'id', 'time', 'y', covariates)

In [5]:
# step a: initial guess
svU, svS, svV = ssla.svds(Y0, K)
svU, svS, svV = np.fliplr(svU), svS[::-1], np.flipud(svV) # reverse the order from largest to smallest
# initial guess for F
F0 = np.diag(svS) @ svV
# initial value for Gama to be updated
Gama0 = np.zeros((L, K))

In [6]:
# K: number of factors
K = 3
# dimensions
N, T, L = X0.shape

vec_len = len(covariates)*K
numer, denom = np.zeros(vec_len), np.zeros((vec_len, vec_len))
for t in range(T):
    for i in range(N_co):
        X0_slice = X0[i, t, :] # X_it is Lx1 vector for each i and t
        Ft_slice = F0[:, t] # F_t is Kx1 vector for each t
        # compute kronecker product
        kron_prod = np.kron(X0_slice, Ft_slice)
        # update numer and denom
        numer += kron_prod * Y0[i, t]
        denom += np.outer(kron_prod, kron_prod)

In [7]:
g = np.array([[2, 3, 4], [1, 2, 3]])
f = np.array([1, 2, 3])
c = np.diag(np.ones(3))

In [8]:
x = np.random.random(10)
g = np.random.random((10, 3))
f = np.random.random(3)
c = np.diag(np.ones(10))

In [9]:
np.kron(c.T, f).shape

(10, 30)

In [10]:
x@np.kron(c, f)

array([0.41933671, 0.32672118, 0.43738613, 0.03977656, 0.03099143,
       0.04148865, 0.29629591, 0.23085541, 0.30904931, 0.30445476,
       0.23721228, 0.31755934, 0.1426832 , 0.1111699 , 0.14882468,
       0.23455386, 0.18274983, 0.24464971, 0.32653576, 0.25441643,
       0.34059076, 0.10630882, 0.08282924, 0.11088465, 0.58460855,
       0.45549075, 0.60977173, 0.5249537 , 0.40901139, 0.54754916])

In [13]:
x@(np.kron(c, f) @ g.reshape(-1, 1))

array([4.07765297])

In [18]:
x

array([0.67921812, 0.06442784, 0.47992353, 0.49313878, 0.23111025,
       0.37991721, 0.52890434, 0.17219307, 0.94691619, 0.8502906 ])